# European Call Option BHP

In [1]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from datetime import datetime

In [3]:
# Pulling close price as at 16 May 2025 from yfinance
ticker = "BHP.AX"
valuation_date = datetime(2025, 5, 16)
expiry_date = datetime(2027, 9, 15)
target_date_str = valuation_date.strftime("%Y-%m-%d")
data = yf.download(ticker, start=target_date_str, end="2027-09-15", progress=False)
if data.empty:
    raise ValueError(f"No data available for {ticker} on {target_date_str}")
S0 = float(data["Close"].iloc[0])  # Ensure scalar

# Calculate time to maturity dynamically 
T = (expiry_date - valuation_date).days / 365.25  # Year fraction


# Pull volatility (sigma) from cell S15 in sheet "Worksheet"
vol_file = "BHP Volatility.xlsx"
from openpyxl import load_workbook

# Open the workbook and access the value from R12
wb = load_workbook(filename=vol_file, data_only=True)
ws = wb["Worksheet"]
sigma = ws["S15"].value  # This returns the evaluated result (not the formula)

if sigma is None:
    raise ValueError("Volatility (sigma) in cell S15 returned None — please check if Excel formula has been calculated and saved.")

wb.close()
print(f"Spot = {S0:.2f}, Volatility = {sigma:.2%}")

Spot = 39.72, Volatility = 20.24%


/tmp/ipykernel_86150/4021668893.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  S0 = float(data["Close"].iloc[0])  # Ensure scalar
